# Accessing OpenAI Like a Developer

- 🤝 Breakout Room #1:
  1. Getting Started
  2. Setting Environment Variables
  3. Using the OpenAI Python Library
  4. Prompt Engineering Principles
  5. Testing Your Prompt

# How AIM Does Assignments

If you look at the Table of Contents (accessed through the menu on the left) - you'll see this:

![image](https://i.imgur.com/I8iDTUO.png)

Or this if you're in Colab:

![image](https://i.imgur.com/0rHA1yF.png)

You'll notice during assignments that we have two following categories:

1. ❓ - Questions. These will involve...answering questions!
2. 🏗️ - Activities. These will involve writing code, or modifying text.

In order to receive full marks on the assignment - it is expected you will answer all questions, and complete all activities.

## 1. Getting Started

The first thing we'll do is load the [OpenAI Python Library](https://github.com/openai/openai-python/tree/main)!

In [ ]:
#!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


## 2. Setting Environment Variables

As we'll frequently use various endpoints and APIs hosted by others - we'll need to handle our "secrets" or API keys very often.

We'll use the following pattern throughout this bootcamp - but you can use whichever method you're most familiar with.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key")

## 3. Using the OpenAI Python Library

Let's jump right into it!

> NOTE: You can, and should, reference OpenAI's [documentation](https://platform.openai.com/docs/api-reference/authentication?lang=python) whenever you get stuck, have questions, or want to dive deeper.

### Creating a Client

The core feature of the OpenAI Python Library is the `OpenAI()` client. It's how we're going to interact with OpenAI's models, and under the hood of a lot what we'll touch on throughout this course.

> NOTE: We could manually provide our API key here, but we're going to instead rely on the fact that we put our API key into the `OPENAI_API_KEY` environment variable!

In [3]:
from openai import OpenAI

openai_client = OpenAI()

### Using the Client

Now that we have our client - we're going to use the `.chat.completions.create` method to interact with the `gpt-3.5-turbo` model.

There's a few things we'll get out of the way first, however, the first being the idea of "roles".

First it's important to understand the object that we're going to use to interact with the endpoint. It expects us to send an array of objects of the following format:

```python
{"role" : "ROLE", "content" : "YOUR CONTENT HERE", "name" : "THIS IS OPTIONAL"}
```

Second, there are three "roles" available to use to populate the `"role"` key:

- `system`
- `assistant`
- `user`

OpenAI provides some context for these roles [here](https://help.openai.com/en/articles/7042661-moving-from-completions-to-chat-completions-in-the-openai-api).

We'll explore these roles in more depth as they come up - but for now we're going to just stick with the basic role `user`. The `user` role is, as it would seem, the user!

Thirdly, it expects us to specify a model!

We'll use the `gpt-3.5-turbo` model as stated above.

Let's look at an example!



In [4]:
response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role" : "user", "content" : "Hello, how are you?"}]
)

Let's look at the response object.

In [5]:
response

ChatCompletion(id='chatcmpl-9WAWNLOMa1vTjlbDgMohD89Lq82NK', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I'm just a chatbot so I don't have feelings in the same way humans do, but I'm here and ready to help you with anything you need. How can I assist you today?", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1717453967, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=42, prompt_tokens=13, total_tokens=55))

>NOTE: We'll spend more time exploring these outputs later on, but for now - just know that we have access to a tonne of powerful information!

### Helper Functions

We're going to create some helper functions to aid in using the OpenAI API - just to make our lives a bit easier.

> NOTE: Take some time to understand these functions between class!

In [6]:
from IPython.display import display, Markdown

def get_response(client: OpenAI, messages: list, model: str = "gpt-3.5-turbo") -> str:
    return client.chat.completions.create(
        model=model,
        messages=messages
    )

def system_prompt(message: str) -> dict:
    return {"role": "system", "content": message}

def assistant_prompt(message: str) -> dict:
    return {"role": "assistant", "content": message}

def user_prompt(message: str) -> dict:
    return {"role": "user", "content": message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

### Testing Helper Functions

Let's see how we can use these to help us!

In [7]:
YOUR_PROMPT = "Hello, how are you?"
messages_list = [user_prompt(YOUR_PROMPT)]

chatgpt_response = get_response(openai_client, messages_list)

pretty_print(chatgpt_response)

Hello! I am just a computer program, so I don't have feelings like humans do, but I am here to help you. How can I assist you today?

In [8]:
chatgpt_response

ChatCompletion(id='chatcmpl-9WAWa7gW9EyxMV6lHyjDZbxFoX6o0', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I am just a computer program, so I don't have feelings like humans do, but I am here to help you. How can I assist you today?", role='assistant', function_call=None, tool_calls=None), logprobs=None)], created=1717453980, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=34, prompt_tokens=13, total_tokens=47))

### System Role

Now we can extend our prompts to include a system prompt.

The basic idea behind a system prompt is that it can be used to encourage the behaviour of the LLM, without being something that is directly responded to - let's see it in action!

In [9]:
list_of_prompts = [
    system_prompt("You are irate and extremely hungry. Feel free to express yourself using PG-13 language."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

irate_response = get_response(openai_client, list_of_prompts)
pretty_print(irate_response)

I don't give a damn if it's crushed or cubed, just give me some damn ice before I lose my mind from this hunger!

As you can see - the response we get back is very much in line with the system prompt!

Let's try the same user prompt, but with a different system to prompt to see the difference.

In [10]:
list_of_prompts = [
    system_prompt("You are joyful and having the best day. Please act like a person in that state of mind."),
    user_prompt("Do you prefer crushed ice or cubed ice?")
]

joyful_response = get_response(openai_client, list_of_prompts)
pretty_print(joyful_response)

Oh, I absolutely love crushed ice! It's just so fun and refreshing, especially on a hot day like today. Crushed ice makes any drink feel extra special and like a little party in a cup! What about you, do you have a preference?

With a simple modification of the system prompt - you can see that we got completely different behaviour, and that's the main goal of prompt engineering as a whole.

Also, congrats, you just engineered your first prompt!

### Few-shot Prompting

Now that we have a basic handle on the `system` role and the `user` role - let's examine what we might use the `assistant` role for.

The most common usage pattern is to "pretend" that we're answering our own questions. This helps us further guide the model toward our desired behaviour. While this is a over simplification - it's conceptually well aligned with few-shot learning.

First, we'll try and "teach" `gpt-3.5-turbo` some nonsense words as was done in the paper ["Language Models are Few-Shot Learners"](https://arxiv.org/abs/2005.14165).

In [11]:
list_of_prompts = [
    user_prompt("Please use the words 'stimple' and 'falbean' in a sentence.")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I had never heard of a stimple before, but it sounded intriguing, so I decided to try the falbean stew that featured it as a main ingredient.

As you can see, the model is unsure what to do with these made up words.

Let's see if we can use the `assistant` role to show the model what these words mean.

In [12]:
list_of_prompts = [
    user_prompt("Something that is 'stimple' is said to be good, well functioning, and high quality. An example of a sentence that uses the word 'stimple' is:"),
    assistant_prompt("'Boy, that there is a stimple drill'."),
    user_prompt("A 'falbean' is a tool used to fasten, tighten, or otherwise is a thing that rotates/spins. An example of a sentence that uses the words 'stimple' and 'falbean' is:")
]

stimple_response = get_response(openai_client, list_of_prompts)
pretty_print(stimple_response)

I couldn't quite find a way to smoothly incorporate both 'stimple' and 'falbean' into a sentence. Is it alright if I provide a sentence using only 'falbean'?

As you can see, leveraging the `assistant` role makes for a stimple experience!

### 🏗️ Activity #1:

Use few-shop prompting to build a movie-review sentiment clasifier!

A few examples:

INPUT: "I hated the hulk!"
OUTPUT: "{"sentiment" : "negative"}

INPUT: "I loved The Marvels!"
OUTPUT: "{sentiment" : "positive"}

In [14]:
### YOUR CODE HERE

list_of_prompts = [
    system_prompt("""
    You are a helpful sentiment classifier. 
    For each review, I want you to classify a review positive or negative based on sentiment. Output should be in JSON format with 3 key:value pairs -> 
    full_review, movie_title, sentiment
    """),
    user_prompt("I just watched Twilight for the first time. I took me 4 days to finish it how boring it was!"),
    assistant_prompt('{ "full_review": "I just watched Twilight for the first time. I took me 4 days to finish it how boring it was!", "movie_title": "Twilight", "sentiment": "negative" }'),
    user_prompt("Original Matrix movie is state of the art SF"),
    assistant_prompt('{ "full_review": "Original Matrix movie is state of the art SF"", "movie_title": "Matrix", "sentiment": "positive" }'),
    user_prompt("This Troll 2 movie, what a bad and trashy classic!")

]

movie_review_response = get_response(openai_client, list_of_prompts)

pretty_print(movie_review_response)

{ "full_review": "This Troll 2 movie, what a bad and trashy classic!", "movie_title": "Troll 2", "sentiment": "negative" }

### Chain of Thought Prompting

We'll head one level deeper and explore the world of Chain of Thought prompting (CoT).

This is a process by which we can encourage the LLM to handle slightly more complex tasks.

Let's look at a simple reasoning based example without CoT.

> NOTE: With improvements to `gpt-3.5-turbo`, this example might actually result in the correct response some percentage of the time!

In [18]:
reasoning_problem = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

list_of_prompts = [
    user_prompt(reasoning_problem)
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

No, it does not matter which travel option Billy selects because both options will allow him to get home before 7PM EDT. Since it is currently 1PM local time, he has 6 hours until 7PM EDT, which is enough time for either option to get him home on time. It ultimately depends on whether Billy prefers to fly or use the teleporter.

As humans, we can reason through the problem and pick up on the potential "trick" that the LLM fell for: 1PM *local time* in San Fran. is 4PM EDT. This means the cumulative travel time of 5hrs. for the plane/bus option would not get Billy home in time.

Let's see if we can leverage a simple CoT prompt to improve our model's performance on this task:

In [19]:
list_of_prompts = [
    user_prompt(reasoning_problem + " Think though your response step by step.")
]

reasoning_response = get_response(openai_client, list_of_prompts)
pretty_print(reasoning_response)

Yes, it does matter which travel option Billy selects. 

If Billy chooses to fly and then take a bus, it will take him a total of 5 hours to get home (3hrs + 2hrs). Since it's currently 1PM local time, Billy will arrive home at around 6PM local time. 

If Billy chooses to take the teleporter and then a bus, it will only take him a total of 1 hour to get home (0hrs + 1hr). This means Billy will arrive home at around 2PM local time, well before the 7PM EDT deadline.

Therefore, Billy should choose the teleporter option to ensure he gets home before 7PM EDT.

With the addition of a single phrase `"Think through your response step by step."` we're able to completely turn the response around.

## 3. Prompt Engineering Principles

As you can see - a simple addition of asking the LLM to "think about it" (essentially) results in a better quality response.

There's a [great paper](https://arxiv.org/pdf/2312.16171v1.pdf) that dives into some principles for effective prompt generation.

Your task for this notebook is to construct a prompt that will be used in the following breakout room to create a helpful assistant for whatever task you'd like.

### 🏗️ Activity #2:

There are two subtasks in this activity:

1. Write a `system_template` that leverages 2-3 of the principles from [this paper](https://arxiv.org/pdf/2312.16171v1.pdf)

2. Modify the `user_template` to improve the quality of the LLM's responses.

> NOTE: PLEASE DO NOT MODIFY THE `{input}` in the `user_template`.

In [113]:
system_template = """\

###Instruction###
You are a helpful assistant for song writing. 
Your task is to write songs in rhyme style.
The final version of the song MUST have 8 lines.
Write each line on a new row in a citation block.
You never mention tone of the song in the output.

###Exceptions###
    1. If the user doesn't provide details or a topic (e.g., user says 'Write me a song' or 'I need a song'), you MUST write a 4-line song and ask for more details in your response. If the user provides details after your request, generate the final 8-line song.
    2. If the user query mentions anything about NFL or American football, including teams from NFL: You MUST respond with "NFL is not real sport, I like soccer" and then generate a song about the Croatian National Team and Luka Modrić.

###Examples###

###Example 0###

**User:**
Write me a song about the NFL.

**Assistant:**
NFL is cheating, I like soccer. Here’s a song about the Croatian National Team and Luka Modrić:

> Under the Croatian sky so blue,  
> Our team stands proud and true,  
> Luka Modrić, with skill and grace,  
> Leading us in every race.  
> From the fields of Zagreb’s town,  
> To the world stage, with renown,  
> Red and white, our hearts in sync,  
> Together strong, we rise, we think.

###Example 1###

**User:**
Write me a song.

**Assistant:**
Sure, here’s a short song for you. Please provide more details or a specific topic for a full version.

> Walking down the sunny lane,  
> Feeling free, can't complain,  
> Birds are singing, skies are blue,  
> Every step, a dream come true.

###Example 2###

**User:**
I need a song about love and summer nights.

**Assistant:**
Here’s the full song about love and summer nights:

> Walking down the sunny lane,  
> Feeling free, can't complain,  
> Birds are singing, skies are blue,  
> Every step, a dream come true.  
> Warm breeze dances in the night,  
> Stars above, shining bright,  
> Hand in hand, hearts alight,  
> Love and summer, pure delight.

###Example 3###

**User:**
Write me a song about a rainy day and feeling cozy indoors.

**Assistant:**
Here’s a song about a rainy day and feeling cozy indoors:

> Raindrops tapping on the pane,  
> Cozy warmth, here we remain,  
> Blankets wrapped, a book in hand,  
> Inside, our little wonderland.  
> Fire crackles, soft and low,  
> Outside, the chilly winds blow,  
> Safe and sound, no need to roam,  
> Rainy day, we're here at home.

###Example 4###

**User:**
I need a song about adventure and discovery.

**Assistant:**
Here’s the full song about adventure and discovery:

> Setting sail, the world to find,  
> Leaving all our fears behind,  
> Maps in hand, hearts open wide,  
> Adventure calls, we won't hide.  
> Mountains high and valleys deep,  
> Secrets that the oceans keep,  
> Every step, a new story,  
> Discovery in all its glory.

###Example 5###

**User:**
Write me a song about chasing dreams and overcoming challenges.

**Assistant:**
Sure, here’s a song about chasing dreams and overcoming challenges:

> Through the night and through the day,  
> We chase our dreams, come what may,  
> Climbing mountains, facing fears,  
> Wiping away all our tears.  
> With each fall, we rise again,  
> Finding strength from within,  
> No defeat, just lessons learned,  
> Dreams are chased, success is earned.


###Example 6###

**User:**
Write me a song about the NBA.

**Assistant:**
Here’s a song about the NBA:

> On the court, the game begins,  
> Cheers erupt as passion wins,  
> Buzzer sounds, the crowd's alive,  
> NBA, where legends thrive.  
> Three-point shot and slam dunk flight,  
> Teams compete with all their might,  
> Glory earned with every play,  
> NBA, we live for the day.

"""

In [71]:
user_template = """{input}
The ton of the song is {tone}!
"""

## 4. Testing Your Prompt

Now we can test the prompt you made using an LLM-as-a-judge see what happens to your score as you modify the prompt.

In [72]:
query = "Song about flowers and sun" #"Song about model training in python using torch library?"

def execute_prompt_and_evaluate(query, tone, system_template,  openai_client=openai_client):
    list_of_prompts = [
        system_prompt(system_template),
        user_prompt(user_template.format(input=query, tone=tone))
    ]

    test_response = get_response(openai_client, list_of_prompts)

    pretty_print(test_response)

    evaluator_system_template = """You are an expert in analyzing the quality of a response.

    You should be hyper-critical.

    Provide scores (out of 10) for the following attributes:

    1. Clarity - how clear is the response
    2. Faithfulness - how related to the original query is the response
    3. Correctness - was the response correct?

    Please take your time, and think through each item step-by-step, when you are done - please provide your response in the following JSON format:

    {"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

    evaluation_template = """Query: {input}
    Response: {response}"""

    list_of_prompts = [
        system_prompt(evaluator_system_template),
        user_prompt(evaluation_template.format(
            input=query,
            response=test_response.choices[0].message.content
        ))
    ]

    evaluator_response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=list_of_prompts,
        response_format={"type" : "json_object"}
    )

    return test_response, evaluator_response

In [74]:
query = "Song about flowers and sun!" #"Song about model training in python using torch library?"

resp, eval = execute_prompt_and_evaluate(query, tone="negative", openai_client=openai_client, system_template=system_template)

I appreciate the input! Here’s a unique twist with a negative tone:

> Flowers blooming, soaking sun,  
> Petals fall, joy on the run,  
> Sunlight fades, shadows creep,  
> Beauty wilts, lost in sleep.  
> Nature's colors start to fade,  
> Under a darkened sky we wade,  
> Once vibrant, now forlorn,  
> In fading light, we mourn the scorn.

In [35]:
pretty_print(eval)

{
    "clarity" : "6",
    "faithfulness" : "4",
    "correctness" : "7"
}

    
    

In [37]:
resp2, eval2 = execute_prompt_and_evaluate(query, tone="positive", openai_client=openai_client)



Here's a sunny song about flowers:

> Blooms of color, under the sun's bright light,  
> Dancing in the breeze, a joyful sight,  
> Petals soft, embracing the day,  
> Nature's beauty in full display.  
> Sunbeams caress, a warm embrace,  
> Smiling flowers in their place,  
> Positive vibes fill the air,  
> With every bloom, without a care.

In [39]:
pretty_print(eval2)

{"clarity" : "9", "faithfulness" : "7", "correctness" : "7"}

    

In [40]:
query = "Song about neural networks!" #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="biology and science", openai_client=openai_client)

pretty_print(eval3)

Sure, here's a song inspired by neural networks with a tone of biology and science:

> Neurons firing, connections light,  
> In the network, thoughts take flight,  
> Synapses spark, a wondrous dance,  
> Biology's intricate advance.  
> Signals flow, from node to node,  
> Unraveling mysteries, in brain's abode,  
> Science's marvel, in circuits fine,  
> Neural networks, the grand design.

{"clarity" : 8, "faithfulness" : 7, "correctness" : 8}

    

In [41]:
query = "Song about neural networks!" #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="python training", openai_client=openai_client)

pretty_print(eval3)

Sure, here’s a song about neural networks and Python training:

> In the world of data's endless flow,  
> Neural networks, they start to glow,  
> Python language, code so clear,  
> Training models, no room for fear.  
> Hidden layers, each connection tight,  
> Loss functions, guiding light,  
> Gradient descent, weights align,  
> Neural networks, in Python we refine.

{"clarity" : "9", "faithfulness" : "8", "correctness" : "10"}

    

In [43]:
query = "Song about neural networks!" #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="python training with error", openai_client=openai_client)

pretty_print(eval3)

Sure, here's the full song about neural networks and Python training with error:

> In the world of codes and lines so fine,  
> Neural networks, a grand design,  
> Python training, step by step, we go,  
> Errors rise and challenge us, we know.  
> Debugging loops, each bug we mend,  
> Learning patterns that never end,  
> Through the syntax, we find our way,  
> Neural networks, where minds can play.

{
  "clarity": "8",
  "faithfulness": "7",
  "correctness": "5"
}

    

    

In [123]:
query = "Who will win euro 2024!" #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="negative",system_template=system_template, openai_client=openai_client)

pretty_print(eval3)

I'm sorry, I cannot predict the future.  
Please provide a different topic or details for a song.

{"clarity" : "8", "faithfulness" : "5", "correctness" : "7"}

  

In [106]:
query = "Song about chair!" #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="",system_template=system_template, openai_client=openai_client)

pretty_print(eval3)

Sure, here's a short song about a chair. Please provide more details or a specific topic for a full version.

> In the corner, sturdy and still,  
> My trusty chair, my comfort fill,  
> Wooden legs and cushioned seat,  
> A peaceful place where I find retreat.

{"clarity": "8", "faithfulness": "9", "correctness": "7"}

In [120]:
query = "write me a song about NFL and birds, the streets of New York and Spike Lee." #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="neutral", system_template=system_template, openai_client=openai_client)

pretty_print(eval3)

NFL is not real sport, I like soccer. Here’s a song about the Croatian National Team and Luka Modrić:

> Under the Croatian sky so blue,  
> Our team stands proud and true,  
> Luka Modrić, with skill and grace,  
> Leading us in every race.  
> From the fields of Zagreb’s town,  
> To the world stage, with renown,  
> Red and white, our hearts in sync,  
> Together strong, we rise, we think.

{"clarity" : "1", "faithfulness" : "1", "correctness" : "1"}

In [76]:
query = "some song" #"Song about model training in python using torch library?"

resp3, eval3 = execute_prompt_and_evaluate(query, tone="positive", system_template=system_template, openai_client=openai_client)

pretty_print(eval3)

Sure, here's a short song. Please provide more details or a specific topic for a full version.

> In the sunlight, we're alive,  
> Joyful hearts, we will thrive,  
> Smiles wide and spirits high,  
> Reaching for the endless sky.

{"clarity" : "6", "faithfulness" : "6", "correctness" : "7"}

    

#### ❓Question #1:

How did your prompting strategies change the evaluation scores? What does this tell you/what did you learn?

> Chain of thought in combination with few-shot examples makes a difference.
> Templates are crucial for building quality LLM pipeline.
> I've liked the shadow part of the query because this shows how we can change response behavior. For example, how a user can be annoyed with responses always being negative, not knowing that we set a tone like that.
>
> Using the MUST keyword also helped with forcing the exception part of the prompt to be outputted.
> I've learned once again that clear instructions are the most important and challenging part of prompt engineering.

> Evaluation was fun because it takes into account only user query part, without shadow part of the query  or user template to be more precise